In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import os
import time
import shutil
import pathlib
import itertools

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.applications import InceptionV3
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [2]:
data_dir = "C:/Users/aniru/OneDrive/Desktop/Capstone/Data"
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)

df.head()


,filepaths,labels
0,C:/Users/aniru/OneDrive/Desktop/Capstone/Data\...,No
1,C:/Users/aniru/OneDrive/Desktop/Capstone/Data\...,No
2,C:/Users/aniru/OneDrive/Desktop/Capstone/Data\...,No
3,C:/Users/aniru/OneDrive/Desktop/Capstone/Data\...,No
4,C:/Users/aniru/OneDrive/Desktop/Capstone/Data\...,No


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1./255)

#img_size = (150, 150)
img_size = (224, 224)

batch_size = 64

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepaths",
    y_col="labels",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepaths",
    y_col="labels",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 808 validated image filenames belonging to 2 classes.
Found 203 validated image filenames belonging to 2 classes.


In [5]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert to dictionary format
class_weights = dict(enumerate(class_weights))

print("Class Weights:", class_weights)


Class Weights: {0: 1.9330143540669857, 1: 0.674457429048414}


In [6]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam

# Build the CNNSVM model
model = models.Sequential()

# First Conv2D layer
model.add(layers.Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Second Conv2D layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Third Conv2D layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Flatten and Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

# Output layer for SVM
model.add(layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.01)))

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='hinge',          # Hinge loss for SVM
    metrics=['accuracy']   # Accuracy as metric
)

# Model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 111, 111, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 55, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 53, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 26, 26, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 128)      0

In [7]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Adam optimizer with a learning rate of 0.0001
    loss='hinge',                        # Hinge loss (used for SVM-like output)
    metrics=['accuracy']                 # Track accuracy during training
)


In [8]:
# Early stopping to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Fit the model
history = model.fit(
    train_generator,          # Training data
    validation_data=test_generator,  # Validation data
    epochs=30,                 # Number of epochs
    batch_size=64,             # Batch size (handled internally by generator)
    callbacks=[early_stopping],
    class_weight=class_weights,# Early stopping callback
    verbose=1                  # Show training progress
)


Epoch 1/30
13/13 [==============================] - 9s 311ms/step - loss: 1.0192 - accuracy: 0.5000 - val_loss: 1.0190 - val_accuracy: 0.5000
Epoch 2/30
13/13 [==============================] - 7s 540ms/step - loss: 1.0188 - accuracy: 0.5000 - val_loss: 1.0186 - val_accuracy: 0.5000
Epoch 3/30
13/13 [==============================] - 7s 547ms/step - loss: 1.0185 - accuracy: 0.5000 - val_loss: 1.0183 - val_accuracy: 0.5000
Epoch 4/30
13/13 [==============================] - 7s 560ms/step - loss: 1.0181 - accuracy: 0.5000 - val_loss: 1.0180 - val_accuracy: 0.5000
Epoch 5/30
13/13 [==============================] - 7s 529ms/step - loss: 1.0178 - accuracy: 0.5000 - val_loss: 1.0176 - val_accuracy: 0.5000
Epoch 6/30
13/13 [==============================] - 8s 605ms/step - loss: 1.0175 - accuracy: 0.5000 - val_loss: 1.0173 - val_accuracy: 0.5000
Epoch 7/30
13/13 [==============================] - 7s 572ms/step - loss: 1.0172 - accuracy: 0.5000 - val_loss: 1.0170 - val_accuracy: 0.5000
Epoch 

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy', fontsize=16)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss', fontsize=16)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.show()

In [ ]:
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

pred_probs = model.predict(test_generator)
pred_labels = np.argmax(pred_probs, axis=1)

report = classification_report(true_labels, pred_labels, target_names=class_labels)
print(report)
